In [1]:
import os
import urllib
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pydub
import speech_recognition  as sr
import numpy as np
from PIL import Image
import time

In [ ]:
driver = webdriver.Chrome(executable_path=r"C:\Users\Saurabh\Downloads\chromedriver_win32\chromedriver.exe")
driver.maximize_window()
driver.get("https://www.google.com/recaptcha/api2/demo")
time.sleep(2)

In [ ]:
driver.save_screenshot("recaptcha_2.png")
frames = driver.find_elements_by_tag_name("iframe")
driver.switch_to.frame(frames[0])
driver.find_element_by_class_name("recaptcha-checkbox-border").click()
driver.switch_to.default_content()
frames = driver.find_element_by_xpath("/html/body/div[2]/div[4]").find_elements_by_tag_name("iframe")
driver.switch_to.frame(frames[0])
time.sleep(2)

In [ ]:
driver.find_element_by_css_selector('button[class="rc-button goog-inline-block rc-button-audio"]').click()
driver.switch_to.default_content()
frames = driver.find_elements_by_tag_name("iframe")
driver.switch_to.frame(frames[-1])
time.sleep(2)


In [ ]:
def image_compare(image_1, image_2):
    arr1 = np.array(image_1)
    arr2 = np.array(image_2)
    if arr1.shape != arr2.shape:
        return False
    maxdiff = np.max(np.abs(arr1 - arr2))
    return maxdiff ==0

def image_compare_file(filename_1, filename_2):
    im1 = Image.open(filename_1)
    im2 = Image.open(filename_2)
    return image_compare(im1, im2)
        

In [ ]:

while(1):
    
    file1='recaptcha.png'
    file2='recaptcha_2.png'
    if image_compare_file(file1, file2):
        print('Completed')
        break
    else:
        src = driver.find_element_by_id("audio-source").get_attribute("src")
        print(src)
        urllib.request.urlretrieve(src, "sample.mp3")
        pydub.AudioSegment.ffmpeg = "ffmpeg.exe"
        sound = pydub.AudioSegment.from_mp3('sample.mp3')
        sound.export("sample.wav", format="wav")


        r = sr.Recognizer()
        sample_audio = sr.AudioFile("sample.wav")
        with sample_audio as source:
            audio = r.record(source)
            key = r.recognize_google(audio)
            print("[INFO] Recaptcha Passcode: %s" % key)
            
        driver.find_element_by_id("audio-response").send_keys(key.lower())
        driver.find_element_by_id("recaptcha-verify-button").click()
        time.sleep(2)
        driver.save_screenshot("recaptcha_2.png")

        #driver.switch_to.default_content()

In [ ]:
driver.find_element_by_css_selector('input[id="recaptcha-demo-submit"]').click()

#driver.find_element_by_id("recaptcha-demo-submit").click()